In [1]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from datetime import datetime
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()
#model_copy = LinearRegressionModel.load('./models/')

In [2]:
def detect(model, current, buzz, stol, last_30m, last_2h, last_4h):
    data = [{"year": datetime.now().year, "month": datetime.now().month, "day": datetime.now().day, "hour": datetime.now().hour, "minute": datetime.now().minute, "second": datetime.now().second, "fill": current, "buzz": buzz , "stol": stol, "fill_past_30m": last_30m, "fill_past_2h": last_2h, "fill_past_4h": last_4h, "fill_future_4h":-1}]
    spark_data = spark.createDataFrame(data)
    feature_cols = ['year', 'month', 'day', 'hour', 'minute', 'second', 'fill', 'buzz', 'stol', 'fill_past_30m', 'fill_past_2h', 'fill_past_4h']
    vect_assembler = VectorAssembler(inputCols = feature_cols, outputCol = "features")
    data_w_features = vect_assembler.transform(spark_data)
    data_for_training = data_w_features.select('features', 'fill_future_4h')
    pred = model.evaluate(data_for_training)
    filtered_preds = pred.predictions.drop("fill_future_4h")
    filtered_preds.show()